In [ ]:
import pandas as pd 
import os 
from dfply import *
import warnings
import pulp
warnings.filterwarnings('ignore')
#place the max columns and max rows to show all the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#loading our wind energy generation data 
project_path = '../'
data_folder_path = os.path.join(project_path, 'data')

In [ ]:
wind_8760 = pd.read_excel(os.path.join(data_folder_path,'wind/KCWF_Energy_Results_22Jun23.xlsx'), sheet_name='8760_clean')

In [ ]:
#loading day ahead pricing data
DA_df = pd.read_excel(os.path.join(data_folder_path,'BEPM_WAUE_LMP_RT_DA_2020_2022_cleaned.xlsx'), sheet_name='Day Ahead')
DA_df['Date']=pd.to_datetime(DA_df['Date'])
DA_df['Year']=DA_df['Date'].dt.year
DA_df['Month']=DA_df['Date'].dt.month
DA_df['Month Name']=DA_df['Date'].dt.month_name()
DA_df['Day']=DA_df['Date'].dt.day
DA_df['Hour']=DA_df['Date'].dt.hour
DA_df['Weekday Name']=DA_df['Date'].map(lambda x: x.strftime('%a'))
DA_df

In [ ]:
DA_df_2020 = DA_df[DA_df['Date'].dt.year==2020].reset_index(drop=True)
DA_df_2020['wind_MWh']=wind_8760['gen_MWh']

DA_df_2021 = DA_df[DA_df['Date'].dt.year==2021].reset_index(drop=True)
DA_df_2021['wind_MWh']=wind_8760['gen_MWh']

DA_df_2022 = DA_df[DA_df['Date'].dt.year==2022].reset_index(drop=True)
DA_df_2022['wind_MWh']=wind_8760['gen_MWh']

DA_df = pd.concat([DA_df_2020,DA_df_2021,DA_df_2022]).reset_index(drop=True)

In [19]:
HYD_CAP_PER_TRUCK_lbs = 2500  # 2500 lbs of hydrogen per truck (SIZE)
LBS_PER_HR = 203  # lbs of hydrogen per hour per electrolyzer
POWER_MW = 5  # MW

# to make it easier to solve we will do only capacity values divisible by 5
# assume it needs to end the year at full charge

# EVALUTATION OF HYDROGEN STORAGE SCENARIOS with OPTIMAL STORAGE MANAGEMENT
res_df_list = []
for YEAR in [2020, 2021, 2022]:
    for tank_count in [1, 2, 3, 4, 5]:

        hyd_capacity_lbs = tank_count * HYD_CAP_PER_TRUCK_lbs
        hyd_capacity_mwh = hyd_capacity_lbs / LBS_PER_HR * POWER_MW

        hyd_capacity_mwh_divisible5 = hyd_capacity_mwh//POWER_MW * POWER_MW #make divisible by 5 

        pricing_df_by_year = (DA_df[(DA_df['Year'] == YEAR)
                                        # & (DA_df['Month'] == MONTH_NUM)
                                        ]
                                .reset_index(drop=True)
                                )

        electricity_prices = pricing_df_by_year['Close'].tolist()
        wind_8760 = pricing_df_by_year['wind_MWh'].tolist()

        # Set up list length & indexing variables
        T = len(electricity_prices)
        l = list(range(T))
        l_plus_1 = list(range(T+1))

        # Create a LP problem
        lp_problem = pulp.LpProblem(
                "Hydrogen_Storage_Optimization", pulp.LpMinimize)

        # Decision Variables
        charge = pulp.LpVariable.dicts("charge", l, cat=pulp.LpBinary)
        discharge = pulp.LpVariable.dicts("discharge", l, cat=pulp.LpBinary)
        idle = pulp.LpVariable.dicts("idle", l, cat=pulp.LpBinary)
        P_grid = pulp.LpVariable.dicts("P_grid", l, cat='Continuous')

        # State of Charge (SoC) of the hydrogen variable at each time step (tracking variable)
        SoC = pulp.LpVariable.dicts("SoC", l_plus_1, cat='Continuous')

        # Objective Function
        lp_problem += pulp.lpSum(electricity_prices[t] * P_grid[t]
                                for t in range(0, T))
        
        # Initial and Final State of Charge (SoC) Constraints
        lp_problem += SoC[0] == hyd_capacity_mwh_divisible5, "SoC_initial_constraint"   # initial charge of the battery (fully charged)
        lp_problem += SoC[T] >= hyd_capacity_mwh_divisible5, "SoC_final_constraint"  


        # Battery State of Charge (SoC) Constraint
        for t in range(0, T):  # all written from the perspective of the initial line, so the decisions made at time t are applied at time t+1
                lp_problem += SoC[t+1] == SoC[t] + 5 * charge[t] - 15 * discharge[t], f"SoC_Constraint_at_{t+1}"  # gives you the next initial

        # Constraints
        for t in range(0, T):
                lp_problem += SoC[t+1] >= 0, f"SoC_non_negative_constraint_at_{t+1}"

        # Maximum Capacity Constraint
        for t in range(0, T):
                lp_problem += SoC[t] + 5 * charge[t] - 15 * discharge[t] <= hyd_capacity_mwh_divisible5, f"Max_Capacity_Constraint_at_{t+1}"

        # Charge-Discharge Mode Constraint
        for t in range(0, T):
                lp_problem += charge[t] + discharge[t] + idle[t] == 1, f"Charge_Discharge_Mode_Constraint_at_{t}"

        # Grid Interaction Constraint
        for t in range(0, T):
                lp_problem += 15 - 15 * discharge[t] + 5 * charge[t] == P_grid[t],f"Grid_activity_constraint_at_{t}"

        # Grid Power Draw Constraint greater than zero (No export to the grid because this isnt' an actual battery)
        for t in range(0, T):
                lp_problem += P_grid[t] >= 0, f"Grid_non_negative_constraint_at_{t}"

        # Solve the LP problem
        lp_problem.solve(pulp.PULP_CBC_CMD(msg=False))

        # Print the results
        print("Status:", pulp.LpStatus[lp_problem.status])
        print("Total Expense:", pulp.value(lp_problem.objective))


        # save the results to a dataframe
        results_df = pd.DataFrame(columns=['Time', 'Grid', 'Initial_SoC',
                                'Final_SoC', 'Charge', 'Discharge', 'Idle', 'Price', 'Year', 'Month'])
        results_df['Time'] = range(0, T)
        results_df['Grid'] = [pulp.value(P_grid[t]) for t in range(0, T)]
        results_df['Initial_SoC'] = [pulp.value(SoC[t]) for t in range(0, T)]
        results_df['Final_SoC'] = [pulp.value(SoC[t+1]) for t in range(0, T)]
        results_df['Charge'] = [pulp.value(charge[t]) for t in range(0, T)]
        results_df['Discharge'] = [pulp.value(discharge[t]) for t in range(0, T)]
        results_df['Idle'] = [pulp.value(idle[t]) for t in range(0, T)]
        results_df['Price'] = [electricity_prices[t] for t in range(0, T)]
        results_df['Year'] = YEAR
        results_df['Month'] = pricing_df_by_year['Month']
        results_df['Capacity-lbs'] = hyd_capacity_lbs
        results_df['Capacity-MWh'] = hyd_capacity_mwh
        res_df_list.append(results_df)

Status: Optimal
Total Expense: 5625.0
